# N body problem

Some notes and tests for n-body problem solution

According to Newton's law a force of attraction between body $i$ and $j$ can be expressed as

$$
\mathbf{F}_{ij}
    = \frac{G m_i m_j}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\| ^ 2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$

Resultant force acting on $i$ can be found when summing over all the interactions with $N-1$ bodies 

$$
\mathbf{F}_{i}
    =  \sum_{j=1 \atop j \ne i}^N \frac{G m_i m_j }{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|^2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$

or in explicit differential form

$$
\frac{d^2\mathbf{r}_i}{dt^2}
    =  \sum_{j=1 \atop j \ne i}^N \frac{G m_j }{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|^2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$




In [125]:
import numpy as np
from scipy.integrate import odeint
from typing import List, Dict, Tuple, Self
from itertools import combinations, chain
from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation

G = 1

In [126]:
class Body:
    def __init__(
        self,
        m : float,
        r_0 : Tuple[float, float], 
        v_0 : Tuple[float, float],
        name: str = "body"
    ) -> None:
        self.m = m
        self.r_0 = r_0
        self.x_0 = r_0[0]
        self.y_0 = r_0[1]
        self.v_0 = v_0
        self.vx_0 = v_0[0]
        self.vy_0 = v_0[1]
        self.name = name

    def distance(self, body: Self) -> float:
        return np.sqrt((self.x_0 - body.x_0) ** 2 + (self.y_0 - body.y_0) ** 2)

earth = Body(
    m = 1, 
    r_0 = (1, 0), 
    v_0 = (0, np.sqrt(333000)),
    name = "earth"
)
sun = Body(
    m = 333000, 
    r_0 = (0, 0),
    v_0 = (0, 0),
    name = "sun"
)
pp = Body(
    m = 100000,
    r_0 = (1, 2),
    v_0 = (0, 0),
    name = "pp"
)

r_sun_earth = sun.distance(earth)
r_sun_earth

1.0